In [ ]:
import os
import hw
import numpy as np
import env_ingredient_add as env
from hw import PoseTrajectorySource
import trajectories as traj

def get_scene():
    if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
        #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
        scene = env.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
    else:
        #scene = open("objects/environment_setup.yaml") # local setup
        scene = env.get_environment_set_up(no_scene=False,include_driver=True)
        xmls = [os.getcwd() + "/package.xml"]

    return scene

In [48]:
scenario = hw.LoadScenario(data=get_scene())
meshcat = hw.StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [ ]:
# at present, trajectory blank doesn't work
diagram,sim = hw.init_diagram(meshcat,scenario)
# hw.print_diagram(diagram)

context = diagram.CreateDefaultContext()
traj0 = hw.create_small_trajectory(diagram,meshcat,context)

meshcat.Delete()
diagram,sim = hw.init_diagram(meshcat,scenario,traj0)
controller = diagram.GetSubsystemByName("diff_ik_integrator")


KeyboardInterrupt: 

In [ ]:
hw.fix_base_pos(controller.get_parameters(), 3*[True])
hw.create_small_trajectory(diagram,meshcat)

hw.run_simulation(meshcat,sim,20)

In [ ]:
# TODO(mattg): move this into some sort of planner system that can move between states.

diagram,sim = hw.init_diagram(meshcat,scenario)
traj1 = traj.traj_linear_move_to_bowl_0(diagram)
meshcat.Delete()

diagram,sim = hw.init_diagram(meshcat,scenario,traj1)

# problem: we're missing a way to actually pass new trajectories into the robot.

# pose_source = diagram.GetSubsystemByName("hw.PoseTrajectorySource@00007fc3283167c0")

In [55]:
hw.run_simulation(meshcat,sim,20)

In [46]:
from pydrake.all import LeafSystem, AbstractValue
from enum import Enum

class State(Enum):
    INITIALIZING = 0
    MOVING_TO_BOWL = 1
    TASK_COMPLETE = 2

class Planner(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        self.state_enum = State
        self.DeclareAbstractInputPort("traj_input", AbstractValue.Make(PoseTrajectorySource()))
        self.DeclareAbstractOutputPort("traj_output", lambda: AbstractValue.Make(PoseTrajectorySource()), self.CopyTrajectory)
        self.DeclareDiscreteState(1)  # State to hold the current state of the state machine
        self.DeclarePeriodicDiscreteUpdateEvent(0.1, 0.0, self.Update)

    def CopyTrajectory(self, context, output):
        traj_input = self.get_input_port(0).Eval(context)
        output.set_value(traj_input)

    def Update(self, context, discrete_state):
        state = self.state_enum(discrete_state.get_mutable_vector().GetAtIndex(0))
        # State machine logic
        if state == self.state_enum.INITIALIZING:
            print("State: Initializing")
            state = self.state_enum.MOVING_TO_BOWL
        elif state == self.state_enum.MOVING_TO_BOWL:
            print("State: Moving to bowl")
            state = self.state_enum.TASK_COMPLETE
        elif state == self.state_enum.TASK_COMPLETE:
            print("State: Task complete")
            state = self.state_enum.INITIALIZING
        discrete_state.get_mutable_vector().SetAtIndex(0, state.value)

## scratch code

### gets the name of the robot's joints

In [ ]:

# plant = diagram.GetSubsystemByName("station").GetSubsystemByName("plant")

# for index in plant.GetJointIndices():
#     print(plant.get_joint(index).name())

iiwa_base_x
iiwa_base_y
iiwa_base_z
iiwa_joint_1
iiwa_joint_2
iiwa_joint_3
iiwa_joint_4
iiwa_joint_5
iiwa_joint_6
iiwa_joint_7
iiwa_joint_ee
tool0_joint
left_finger_sliding_joint
right_finger_sliding_joint
iiwa_link_7_welds_to_body


In [ ]:


print(controller.get_parameters().get_joint_position_limits())

print(f"after running: {controller.get_parameters().get_joint_position_limits()}")

hw.run_simulation(meshcat,sim,10)